In [94]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import torch
#import torchtext
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils
from torchvision.transforms import Resize, ToTensor, Normalize
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler



In [95]:
train_path = "/opt/ml/input/data/train/"
train_image_dir_path = os.path.join(train_path, 'images')

In [96]:
# Dataset 생성


In [97]:
def search(dirname, result): #하위 목록의 모든 파일을 찾는 함수
    try:
        filenames = os.listdir(dirname)
        for filename in filenames:
            if filename[0] == '.': #.으로 시작하는 애들 거름
                continue
            full_filename = os.path.join(dirname, filename)
            if os.path.isdir(full_filename):
                search(full_filename, result)
            else:
                ext = os.path.splitext(full_filename)[-1]# 확장자 체크
                if ext:
                    result.append(full_filename)
    except PermissionError:
        pass

In [98]:
all_path = []
search(train_image_dir_path, all_path)

In [99]:
# train 데이터의 디렉토리는 2700개.

In [100]:
len(all_path) #2700

18900

In [101]:
all_path[:10]

['/opt/ml/input/data/train/images/006451_female_Asian_18/mask3.jpg',
 '/opt/ml/input/data/train/images/006451_female_Asian_18/normal.jpg',
 '/opt/ml/input/data/train/images/006451_female_Asian_18/mask5.jpg',
 '/opt/ml/input/data/train/images/006451_female_Asian_18/mask4.jpg',
 '/opt/ml/input/data/train/images/006451_female_Asian_18/incorrect_mask.jpg',
 '/opt/ml/input/data/train/images/006451_female_Asian_18/mask2.jpg',
 '/opt/ml/input/data/train/images/006451_female_Asian_18/mask1.jpg',
 '/opt/ml/input/data/train/images/004241_male_Asian_60/mask3.jpg',
 '/opt/ml/input/data/train/images/004241_male_Asian_60/normal.jpg',
 '/opt/ml/input/data/train/images/004241_male_Asian_60/mask5.jpg']

In [102]:
exts = []
for i in all_path:
    ext = os.path.splitext(i)[-1]
    if ext not in exts:
        exts.append(ext)
print(exts)

['.jpg', '.jpeg', '.png']


In [103]:
all_path = sorted(all_path)


In [104]:
#라벨링 하는 함수

In [105]:
def labeling(name):
    label = 0
    info, mask_type = name.split('/')[-2:]
    info = info.split('_')
    gender, age = info[1], int(info[3])
    if 'incorrect' in mask_type:
        label += 6
    elif 'normal' in mask_type:
        label += 12
    
    if gender == 'female':
        label += 3
    
    if 30<= age < 60:
        label += 1
    elif age >= 60:
        label += 2
    return label


In [106]:
#path, label을 컬럼으로 갖는 dataframe 생성

In [107]:
train_path_label_df = pd.DataFrame(all_path, columns = ['path'])

train_path_label['label'] = train_path_label_df['path'].map(lambda x : labeling(x))
train_path_label

,path,label
0,/opt/ml/input/data/train/images/000001_female_...,10
1,/opt/ml/input/data/train/images/000001_female_...,4
2,/opt/ml/input/data/train/images/000001_female_...,4
3,/opt/ml/input/data/train/images/000001_female_...,4
4,/opt/ml/input/data/train/images/000001_female_...,4
...,...,...
18895,/opt/ml/input/data/train/images/006959_male_As...,0
18896,/opt/ml/input/data/train/images/006959_male_As...,0
18897,/opt/ml/input/data/train/images/006959_male_As...,0
18898,/opt/ml/input/data/train/images/006959_male_As...,0


In [108]:
# train_path_label.to_csv('./train_path_label.cvs', index=False, encoding='utf-8')
#train_path_label = pd.read_csv('./train_path_label.csv, econding='utf-8')


In [109]:
#dataset을 상속받은 CustomDataset
# transform 은 size를 512 384로 변형, tensor, 정규화

In [110]:
class CustomDataset(Dataset):
    def __init__(self, img_paths_label, transform):
        self.X = img_paths_label['path']
        self.Y = img_paths_label['label']
        self.transform = transform
        
    def __getitem__(self, index):
        image = Image.open(self.X.iloc[index])
        label = self.Y.iloc[index]
        
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)
    def __len__(self):
        return len(self.X)
    

In [111]:
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2))
])

In [112]:
#train, valid를 나누는 부분
#label의 비율을 유지하면서 나눔


In [113]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train_path_label, test_size=0.2, 
                               shuffle = True, stratify=train_path_label['label'], 
                               random_state = 34)

In [114]:
train.shape, valid.shape

((15120, 2), (3780, 2))

In [115]:
BATCH_SIZE = 64

In [116]:
train_dataset = CustomDataset(train, transform)

train_dataloader = DataLoader(train_dataset, 
                             batch_size=BATCH_SIZE,
                             shuffle=True
                             )


In [117]:
valid_dataset = CustomDataset(valid, transform)

valid_dataloader = DataLoader(valid_dataset, 
                             batch_size=BATCH_SIZE,
                             shuffle=True
                             )


In [118]:
#dataloader는 (batch_size, channell, height, wide)를 출력해줍니다

In [119]:
next(iter(train_dataloader))[0].shape

torch.Size([64, 3, 512, 384])

In [120]:
#모델
# pretrained 된 resnet18사용
#마지막 fc층만 18개의 class로 변경

In [121]:
resnet18 = torchvision.models.resnet18(pretrained=True)

In [129]:
import math
OUTPUT_CLASS_NUM = 18
resnet18.fc = torch.nn.Linear(in_features = 512, out_features= OUTPUT_CLASS_NUM, bias = True) # num of ouput = 18

# xavier uniform
torch.nn.init.xavier_uniform_(resnet18.fc.weight)
stdv = 1. / math.sqrt(resnet18.fc.weight.size(1))
resnet18.fc.bias.data.uniform_(-stdv, stdv)

resnet18.fc.weight.shape[0]



18

In [130]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [131]:
#epoch=5, lr=0.0001


In [168]:
resnet18.to(device)

LEARNING_RATE = 0.0001
NUM_EPOCH = 5

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet18.parameters(), lr=LEARNING_RATE)

dataloaders = {
    "train" : train_dataloader,
    "test" : valid_dataloader
}


In [ ]:
best_test_accuracy = 0.
best_test_loss = 9999.

for epoch in range(NUM_EPOCH):
    for phase in ["train", "test"]:
        running_loss = 0.
        running_acc = 0.
        if phase == "train":
            resnet18.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
        elif phase == "test":
            resnet18.eval() # 네트워크 모델을 eval 모드로 두어 여러 sub module들이 eval mode로 작동할 수 있게 함
        
        for ind, (images, labels) in enumerate(dataloaders[phase]):
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함
            
            with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
                logits = resnet18(images)
                _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9, 1.2, 3.2, 0.1, -0.1,...])을 최대 ouput index를 찾아 예측 레이블 ([2])로 변경함
                loss = loss_fn(logits, labels)
                
                if phase =="train":
                    loss.backward() # 모델의 예측 값과 실제 값의 CrossEntrophy 차이를 통해 gradient 계산
                    optimizer.step() # 계산된 gradient를 가지고 모델 업데이트
            
            running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
            running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장
        
        # 한 epoch이 모두 종료되었을 때,
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)
        print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
        if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
            best_test_accuracy = epoch_acc
        if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 떄, best loss 계산
            best_test_loss = epoch_loss
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")
            

현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.022, 평균 Accuracy : 0.994
현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.064, 평균 Accuracy : 0.981


In [ ]:
def func_eval(model, data_iter, device):
    with torch.no_grad():
        n_total, n_correct = 0, 0
        model.eval()
        for batch_in, batch_out in data_iter:
            y_trgt = batch_out.to(device)
            model_pred = model.forward(batch_in.to(device))
            _, y_pred = torch.max(model_pred, 1) # 행으로 비교
            n_correct += (y_pred == y_trgt).sum().item()
            n_total += batch_in.size(0)
        val_accr = (n_correct/n_total)
        #model.train()
    return val_accr

In [ ]:
print(func_eval(resnet18, valid_dataloader, device))

In [ ]:
def check_eval(raw_data, dataloader, model, device):
    result = []
    with torch.no_grad():
        model.eval()
        for i, (X,y) in enumerate(dataloader):
            model_pred = model.forward(X.to(device))
            _, y_pred = torch.max(model_pred, 1)
            
            result.append([valid.iloc[i]['path'], y_pred.cpu().numpy()[0], y.cpu().numpy()[0]])
    result = pd.DataFrame(result, columns=['path', 'pred', 'target'])
    return result

In [ ]:
valid_testing_dataloader = DataLoader(valid_dataset, shuffle = False)
check_eval_df = check_eval(valid, valid_testing_dataloader, resnet18, device)
check_eval_df

In [ ]:
wrong_df = check_eval_df[check_eval_df['pred'] != check_eval_df['target']]
wrong_df = wrong_df.reset_index(drop=True)
wrong_df

In [165]:
'''
def draw_(df):
    plt.figure(figsize = (15,30))
    row = 7 # wrong df의 개수에 따라 조정할 것
    for i in range(df.shape[0]):
        plt.subplot(row + 1, df.shape[0]//row, i+1)
        plt.imshow(Image.open(df['path'][i]))
        plt.title(f"target:{df['target'][i]}, pred:{df['pred'][i]}", color = 'r', size=20)
        plt.axis('off')
    plt.tight_layout()
    plt.show()
    
'''

In [ ]:
# draw_(wrong_df)